In [ ]:
"""
Purpose: To visualize how the proofreading performs on the manually proofread cells

Pseucode: 
For the example neuron id
1) get the neuron decomposition
2) Run the error detection to get the errored faces
3) Get the errored synapses
4) Plot them mesh and errored synapses



"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path
import neuron_visualizations as nviz


import datajoint_utils as du
from importlib import reload

INFO - 2020-11-30 23:23:29,922 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-30 23:23:29,923 - settings - Setting database.user to celiib
INFO - 2020-11-30 23:23:29,923 - settings - Setting database.password to newceliipass
INFO - 2020-11-30 23:23:29,943 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-30 23:23:29,944 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 23:23:29,963 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-30 23:23:30,317 - settings - Setting enable_python_native_blobs to True


In [2]:
import time
import datajoint_utils as du
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

from datajoint_utils import *

import error_detection as ed

minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-30 23:23:30,336 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 23:23:30,605 - settings - Setting enable_python_native_blobs to True


In [3]:
m65 = minnie
decimation_version = 0
decimation_ratio = 0.25
key_source = (m65.Decimation().proj(decimation_version='version')  &
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)
              & m65.ManualProofreadIDs() & (m65.BaylorSegmentCentroid() & "multiplicity=1").proj())
nucleus_table =(m65.NucleusID & 'nucleus_version=1')  # Aug 1 timestamp
proofread_nucs = nucleus_table & m65.ManualProofreadIDs()
single_soma_nucs = nucleus_table & (dj.U('segment_id').aggr(proofread_nucs, n_nucs='count(nucleus_id)') & 'n_nucs=1').proj()
final_nucs = key_source* single_soma_nucs
final_nucs

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),nucleus_id id of segmented nucleus,nucleus_version segmentation version,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
864691135564655959,0,0.25,225498,1,149712,151648,17549,1315328,85369450370640387
864691135748568361,0,0.25,230236,1,162848,184560,19809,1385955,87203504753932217
864691136194042326,0,0.25,258307,1,174944,137648,21113,1587826,88815663879212610
864691135233108569,0,0.25,262773,1,168096,161952,21503,1314296,87904100020764569
864691135740225387,0,0.25,264870,1,164832,177856,21208,1831649,87413717834430502
864691136105498585,0,0.25,269247,1,177536,213824,19894,1571057,89177746601154881
864691134988386682,0,0.25,269380,1,168016,216288,21534,1688056,87911453071322695
864691136333776819,0,0.25,271518,1,168464,226192,21121,1666190,87912758674384086
864691135771629819,0,0.25,294545,1,182976,135728,21116,1462702,89941288908168255
864691135748575017,0,0.25,296726,1,180976,143888,22162,1827886,89660913577301286


In [4]:
eligible_table = minnie.Decomposition & final_nucs.proj()

In [5]:
seg_ids_to_test = eligible_table.fetch("segment_id")
seg_ids_to_test

array([864691135564655959, 864691135748568361, 864691136194042326,
       864691135233108569, 864691135740225387, 864691136105498585,
       864691134988386682, 864691136333776819, 864691135771629819,
       864691136105484249, 864691135212632448, 864691136296650011,
       864691136311740477, 864691135491233631, 864691135945413668,
       864691135393964789, 864691135113167769, 864691136105493209,
       864691135272164113, 864691136618403213, 864691135974454639,
       864691136003930314, 864691135510455760, 864691136537220258,
       864691135785277636, 864691136333787571, 864691136105491417,
       864691135645592260, 864691136056391384, 864691136099807093,
       864691135348272855, 864691135737374100, 864691135212690816,
       864691136550489250, 864691136194103510, 864691135589907979,
       864691135407289801, 864691136545546146, 864691135382947307,
       864691136008425132, 864691135925564174, 864691136333790899,
       864691135974454895, 864691136008573614, 864691136134446

# Doing the Actual Function

In [6]:
segment_id = seg_ids_to_test[4]
key = dict(segment_id=segment_id)
print(key)

{'segment_id': 864691135740225387}


In [7]:
segment_id = key["segment_id"]
global_start = time.time()

verbose = True

print(f"\n\n----- Working on {segment_id}-------")
whole_pass_time = time.time()

neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")

print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")



----- Working on 864691135740225387-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f7c48df47b8>, <function median_mesh_center at 0x7f7c48df4a60>, <function n_spines at 0x7f7c48df48c8>, <function n_faces_branch at 0x7f7c49470488>, <function skeleton_distance_branch at 0x7f7c48df4840>, <function spines_per_skeletal_length at 0x7f7c48df4d08>, <function no_spine_median_mesh_center at 0x7f7c48df4b70>]
functions_list = [<function axon_segment at 0x7f7c48df6400>]
Axons not keeping because of soma: {'L1': array([ 6,  7,  8,  9, 12, 13])}

----- Working on L0 ------
-- Axon Group 0 of size 29--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 1 of size 1--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 2 of size 1--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- 

In [8]:
errored_faces,axon_faces,non_axon_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,
                                                                  visualize_errors_at_end=False,
                                                                 return_axon_non_axon_faces=True)

functions_list = [<function width at 0x7f7c48df47b8>, <function median_mesh_center at 0x7f7c48df4a60>, <function n_spines at 0x7f7c48df48c8>, <function n_faces_branch at 0x7f7c49470488>, <function skeleton_distance_branch at 0x7f7c48df4840>, <function spines_per_skeletal_length at 0x7f7c48df4d08>, <function no_spine_median_mesh_center at 0x7f7c48df4b70>]
functions_list = [<function axon_segment at 0x7f7c48df6400>]
Axons not keeping because of soma: {'L1': array([ 6,  7,  8,  9, 12, 13])}

----- Working on L0 ------
-- Axon Group 0 of size 29--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 1 of size 1--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 2 of size 1--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 3 of size 5--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 

In [11]:
axon_mesh = neuron_obj.mesh.submesh([axon_faces],append=True)
non_axon_mesh = neuron_obj.mesh.submesh([non_axon_faces],append=True)

nviz.plot_objects(meshes=[axon_mesh,non_axon_mesh],
                 meshes_colors=["red","black"],
                 mesh_alpha=1)

# Visualize the Actual Neuron

In [12]:
print(neuron_obj.segment_id)
nviz.plot_objects(main_mesh=neuron_obj.mesh,
             mesh_alpha=1)

864691135740225387


# Visualize the Results

In [13]:
print(f"n_errored_faces = {len(errored_faces)}")

n_errored_faces = 49496


In [14]:
ed = reload(ed)


if neuron_obj is None:
    neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")

current_mesh = neuron_obj.mesh

error_submesh = current_mesh.submesh([errored_faces],append=True)
valid_mesh = tu.subtract_mesh(current_mesh,error_submesh)

In [15]:
nviz.plot_objects(main_mesh=valid_mesh,
             meshes=error_submesh,
              meshes_colors="red",
             mesh_alpha=1)

# How to get the percentage of synapses removed

In [16]:
"""
Pseudocode: 
1) Get the synapse file that corresponds to it
2) Get the matching v7 id
3) Get the v7 synapses (pre and post) that are still there
4) Get the pre and post that are stil their after autoproofreading
5) Things to find:
- Eliminated and should have been (true positives)
- Eliminated but SHOULDN'T (false positives) 
- Not Eliminated and should have been not (true negative)
- Not Eliminated but should have been eliminated (false negatie) 

"""

"\nPseudocode: \n1) Get the synapse file that corresponds to it\n2) Get the matching v7 id\n3) Get the v7 synapses (pre and post) that are still there\n4) Get the pre and post that are stil their after autoproofreading\n5) Things to find:\n- Eliminated and should have been (true positives)\n- Eliminated but SHOULDN'T (false positives) \n- Not Eliminated and should have been not (true negative)\n- Not Eliminated but should have been eliminated (false negatie) \n\n"

In [17]:
# 1) Get the synapse file that corresponds to it
data_path = Path("./manual_proofread_data/")
current_file = data_path / Path(f"{segment_id}.pkl")
current_file

PosixPath('manual_proofread_data/864691135740225387.pkl')

In [19]:
import system_utils as su
curr_synapse_table = su.load_object(current_file)
#eliminating the self connections
df_pre_filt = curr_synapse_table[curr_synapse_table["pre_pt_root_id_Aug"] != curr_synapse_table["post_pt_root_id_Aug"]]
df_pre_filt

,id,valid,pre_pt_position,pre_pt_supervoxel_id,pre_pt_root_id,ctr_pt_position,post_pt_position,post_pt_supervoxel_id,post_pt_root_id,size,pre_pt_root_id_Aug,post_pt_root_id_Aug
0,74367625,True,"[129888, 204022, 22714]",82632148124941862,864691135581639021,"[129906, 204010, 22717]","[129872, 204028, 22718]",82632148124954543,864691135058569627,236.0,8.646911e+17,8.646911e+17
1,74367860,True,"[130232, 204084, 22678]",82702516869082631,864691135581639021,"[130252, 204148, 22680]","[130306, 204182, 22685]",82702516869083800,864691136911681009,12012.0,8.646911e+17,8.646911e+17
2,84520779,True,"[130948, 203778, 22628]",82772816893726946,864691135581639021,"[130930, 203816, 22630]","[130934, 203850, 22632]",82772816893732334,864691132784239882,3284.0,8.646911e+17,8.646911e+17
3,78498590,True,"[134054, 203094, 22498]",83194960572609323,864691135581639021,"[134086, 203078, 22494]","[134050, 203156, 22486]",83194960572599439,864691136827218542,1652.0,8.646911e+17,8.646911e+17
4,83921928,True,"[134316, 182582, 15810]",83262579665287514,864691135864764636,"[134343, 182606, 15804]","[134384, 182622, 15800]",83262579665279195,864691136050856947,872.0,8.646911e+17,8.646911e+17
...,...,...,...,...,...,...,...,...,...,...,...,...
201,215374443,True,"[209912, 168554, 16454]",93604861047846413,864691136105183863,"[209900, 168580, 16457]","[209830, 168658, 16459]",93604929767341297,864691137197005377,2004.0,8.646911e+17,8.646911e+17
202,214539925,True,"[209734, 184976, 19378]",93607060407073900,864691136333747635,"[209710, 185028, 19384]","[209668, 185070, 19385]",93607129126570249,864691137197005377,1000.0,8.646911e+17,8.646911e+17
203,215374827,True,"[210448, 168846, 16417]",93675298511464573,864691135711530241,"[210444, 168904, 16414]","[210418, 168918, 16414]",93675298511457661,864691137197005377,24244.0,8.646911e+17,8.646911e+17
204,215375986,True,"[210550, 168830, 16447]",93745667255664012,864691134732960264,"[210558, 168792, 16452]","[210488, 168762, 16448]",93745667255667425,864691137197005377,1084.0,8.646911e+17,8.646911e+17


In [32]:
from pykdtree.kdtree import KDTree
# -------------- New Addittion: want to filter the synapses for only those on non-axonal faces ---- #

#A) Get the synapse data for the august neuron 
ctr_pt_position = df_pre_filt["ctr_pt_position"].to_numpy()
synapse_ids = df_pre_filt["id"].to_numpy()
synapse_centers = np.vstack(ctr_pt_position)
synapse_centers_scaled = synapse_centers* [4, 4, 40]
synapse_ids.shape,synapse_centers.shape

#B) Filter to only those ids that are close to 


current_mesh = neuron_obj.mesh
neuron_kd = KDTree(current_mesh.triangles_center)
dist,closest_face = neuron_kd.query(synapse_centers_scaled)

neuron_mesh_labels = np.zeros(len(current_mesh.faces))
neuron_mesh_labels[axon_faces] = 1
closest_face_labels = neuron_mesh_labels[closest_face]

not_considered_synapses_idx = np.where(closest_face_labels==1)[0]
not_considered_synapses_idx



array([   0,    1,    2,    3,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   20,   21,   22,   23,   24,
         26,   27,   28,   29,   30,   31,   32,   33,   34,   35,   36,
         37,   38,   39,   40,   41,   42,   43,   44,   45,   46,   47,
         48,   49,   50,   51,   52,   53,   54,   55,   56,   68,   69,
         70,   71,   72,   73,   74,   76,   77,   78,   79,   80,   81,
         82,   83,   84,   85,   86,   87,   88,   89,   90,   91,   92,
         93,   94,   95,   96,   97,   99,  100,  101,  102,  103,  104,
        105,  106,  108,  109,  111,  112,  113,  123,  127,  128,  129,
        130,  131,  133,  134,  135,  136,  137,  138,  139,  140,  141,
        142,  143,  150,  151,  152,  159,  165,  169,  173,  185,  186,
        187,  188,  192,  193,  194,  197,  198,  199,  201,  205,  206,
        207,  208,  209,  210,  211,  212,  213,  214,  215,  216,  217,
        218,  219,  221,  223,  224,  225,  226,  2

# Visualize the Cancelled Synapses

In [33]:
nviz.plot_objects(meshes=[axon_mesh,non_axon_mesh],
                  meshes_colors=["red","black"],
                 scatters=[synapse_centers_scaled[not_considered_synapses_idx].reshape(-1,3).reshape(-1,3)],
                 scatters_colors="red")

# Resetting the table to only include the non-axon synapses

In [34]:
df = df_pre_filt[~df_pre_filt.id.isin(synapse_ids[not_considered_synapses])]
df

,id,valid,pre_pt_position,pre_pt_supervoxel_id,pre_pt_root_id,ctr_pt_position,post_pt_position,post_pt_supervoxel_id,post_pt_root_id,size,pre_pt_root_id_Aug,post_pt_root_id_Aug
4,83921928,True,"[134316, 182582, 15810]",83262579665287514,864691135864764636,"[134343, 182606, 15804]","[134384, 182622, 15800]",83262579665279195,864691136050856947,872.0,8.646911e+17,8.646911e+17
18,122630591,True,"[157968, 92510, 21197]",86487448007465779,864691135117860061,"[158036, 92466, 21190]","[158098, 92510, 21188]",86487448007456413,864691136460649700,7936.0,8.646911e+17,8.646911e+17
19,132082418,True,"[159300, 125294, 21167]",86702952286479151,864691136951664863,"[159284, 125352, 21160]","[159270, 125424, 21155]",86702952286469405,864691135476161192,4604.0,8.646911e+17,8.646911e+17
25,143737302,True,"[167714, 174378, 22327]",87835449397450476,864691136951664863,"[167789, 174346, 22326]","[167834, 174372, 22330]",87835449397453110,864691134677566408,8808.0,8.646911e+17,8.646911e+17
58,196069947,True,"[197006, 166954, 19788]",91845436687893349,864691136031798459,"[197040, 166942, 19795]","[197058, 167028, 19795]",91845436687898133,864691135888529033,2704.0,8.646911e+17,8.646911e+17
...,...,...,...,...,...,...,...,...,...,...,...,...
198,215374456,True,"[209956, 168678, 16498]",93675298511558919,864691135058570907,"[209933, 168654, 16504]","[209874, 168644, 16504]",93604861047897947,864691137197005377,5676.0,8.646911e+17,8.646911e+17
201,215374443,True,"[209912, 168554, 16454]",93604861047846413,864691136105183863,"[209900, 168580, 16457]","[209830, 168658, 16459]",93604929767341297,864691137197005377,2004.0,8.646911e+17,8.646911e+17
203,215374827,True,"[210448, 168846, 16417]",93675298511464573,864691135711530241,"[210444, 168904, 16414]","[210418, 168918, 16414]",93675298511457661,864691137197005377,24244.0,8.646911e+17,8.646911e+17
204,215375986,True,"[210550, 168830, 16447]",93745667255664012,864691134732960264,"[210558, 168792, 16452]","[210488, 168762, 16448]",93745667255667425,864691137197005377,1084.0,8.646911e+17,8.646911e+17


In [38]:
#1b) Get the synapse data for the august neuron 
ctr_pt_position = df["ctr_pt_position"].to_numpy()
synapse_ids = df["id"].to_numpy()
synapse_centers = np.vstack(ctr_pt_position)
synapse_ids.shape,synapse_centers.shape
timestamps = np.zeros(len(synapse_ids))

In [39]:
# 2) Get the matching v7 id
import pandas as pd
aug_to_v7_table = pd.read_csv("./proofread_cells_seg_correspondence_Aug1-v7.csv")
v7_id = aug_to_v7_table[aug_to_v7_table["old_seg_id"]==segment_id]["new_seg_id"].tolist()[0]
v7_id

864691136951664863

In [40]:
# check that the new v7 segment id is in the synapse table

def get_pd_id_data(col_name,return_unique=False):
    ret_data = df [col_name].to_numpy()
    if return_unique:
        return np.unique(ret_data)
    else:
        return ret_data
    
pre_pt_root_id = get_pd_id_data("pre_pt_root_id")
pre_pt_root_id.shape,v7_id in pre_pt_root_id

((2162,), True)

In [41]:
post_pt_root_id = get_pd_id_data("post_pt_root_id")
post_pt_root_id.shape,v7_id in post_pt_root_id

((2162,), True)

In [42]:
original_presyn = df[(df["pre_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()
original_postsyn = df[(df["post_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()

original_presyn.shape,original_postsyn.shape

((109,), (2053,))

In [43]:
#3) Get the v7 synapses (pre and post) that are still there
manual_remaining_presyn = df[(df["pre_pt_root_id"] == v7_id)]["id"].to_numpy()
manual_remaining_postsyn = df[(df["post_pt_root_id"] == v7_id)]["id"].to_numpy()
manual_remaining_presyn.shape,manual_remaining_postsyn.shape

((39,), (2037,))

In [44]:
# find the non-remaining presyns and postsyns
manual_error_presyn = np.setdiff1d(original_presyn,manual_remaining_presyn)
manual_error_postsyn = np.setdiff1d(original_postsyn,manual_remaining_postsyn)

manual_error_presyn.shape,manual_error_postsyn.shape

((70,), (16,))

In [45]:
len(manual_remaining_presyn) + len(manual_error_presyn), len(manual_remaining_postsyn) + len(manual_error_postsyn)

(109, 2053)

In [46]:
#4) Get the pre and post that are stil their after autoproofreading
ed = reload(ed)
err_synapse_ids,non_err_synapse_ids = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                     errored_faces,return_synapse_ids=True,
                                                                   synapse_centers=synapse_centers,
                                                                   synapse_ids=synapse_ids
                                                                  )

Returning the synpase ids 1) errored synpases 2) non-errored synapses


In [47]:
# divided the valid synapses into presyn and postsyn
auto_remaining_presyn = np.intersect1d(original_presyn,non_err_synapse_ids)
auto_remaining_postsyn = np.intersect1d(original_postsyn,non_err_synapse_ids)

#dividing the error synapses into presyn and postsyn
auto_error_presyn = np.intersect1d(original_presyn,err_synapse_ids)
auto_error_postsyn = np.intersect1d(original_postsyn,err_synapse_ids)

len(auto_remaining_presyn), len(auto_error_presyn), len(auto_remaining_postsyn), len(auto_error_postsyn)

(42, 67, 2002, 51)

# Calculating the final scores

In [48]:
# doing the presyn analysis
true_positive_presyn = np.intersect1d(manual_error_presyn,auto_error_presyn) #deemed an error and was an error
false_positive_presyn = np.intersect1d(manual_remaining_presyn,auto_error_presyn)#deemed an error but not an error

true_negative_presyn = np.intersect1d(manual_remaining_presyn,auto_remaining_presyn) #not deemed an error and should not be an error
false_negative_presyn = np.intersect1d(manual_error_presyn,auto_remaining_presyn) #not deemed and error but should be an error

len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)

(64, 3, 36, 6)

In [49]:
np.sum([len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)])

109

In [50]:
# Doing the postsyn analysis
true_positive_postsyn = np.intersect1d(manual_error_postsyn,auto_error_postsyn) #deemed an error and was an error
false_positive_postsyn = np.intersect1d(manual_remaining_postsyn,auto_error_postsyn)#deemed an error but not an error

true_negative_postsyn = np.intersect1d(manual_remaining_postsyn,auto_remaining_postsyn) #not deemed an error and should not be an error
false_negative_postsyn= np.intersect1d(manual_error_postsyn,auto_remaining_postsyn) #not deemed and error but should be an error

len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)

(1, 50, 1987, 15)

In [51]:
np.sum([len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)])

2053

In [52]:
#
true_positive_total = list(true_positive_presyn) + list(true_positive_postsyn) 
false_positive_total = list(false_positive_presyn) + list(false_positive_postsyn)

true_negative_total = list(true_negative_presyn) + list(true_negative_postsyn)
false_negative_total = list(false_negative_presyn) + list(false_negative_postsyn)

len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)

(65, 53, 2023, 21)

In [53]:
np.sum([len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)])

2162

In [54]:
def precision(TP,FP):
    if (TP + FP) > 0:
        return TP/(TP + FP)
    else:
        return np.nan
    
def recall(TP,FN):
    if (TP + FN) > 0:
        return TP/(TP + FN)
    else:
        return np.nan
    
def f1(TP,FP,FN):
    curr_prec = precision(TP,FP)
    curr_recall = recall(TP,FN)
    
    if curr_prec + curr_recall > 0:
        return 2*(curr_prec*curr_recall)/(curr_prec + curr_recall)
    else:
        return np.nan
    
def calculate_scores(TP,FP,FN):
    return dict(precision=precision(TP,FP),
               recall=recall(TP,FN),
               f1=f1(TP,FP,FN))

In [55]:
# calculating the accuracy, precision and f1

#precision = (true positives)/(true positives + false positives)
presyn_scores =  calculate_scores(len(true_positive_presyn),len(false_positive_presyn),len(false_negative_presyn))
presyn_scores

{'precision': 0.9552238805970149,
 'recall': 0.9142857142857143,
 'f1': 0.9343065693430657}

In [56]:
postsyn_scores =  calculate_scores(len(true_positive_postsyn),len(false_positive_postsyn),len(false_negative_postsyn))
postsyn_scores

{'precision': 0.0196078431372549, 'recall': 0.0625, 'f1': 0.029850746268656716}

In [57]:
total_scores =  calculate_scores(len(true_positive_total),len(false_positive_total),len(false_negative_total))
total_scores

{'precision': 0.5508474576271186,
 'recall': 0.7558139534883721,
 'f1': 0.6372549019607844}

# If no automatic proofreading was done

# Visualize the Results

In [58]:

err_synapses,non_err_synapses = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                     errored_faces,return_synapse_centroids=True,
                                                                   synapse_centers=synapse_centers,
                                                                   synapse_ids=synapse_ids
                                                                  )

print(f"Segment ID = {segment_id}")
print(f"n_synapses = {len(err_synapses) + len(non_err_synapses)}, n_errored_synapse = {len(err_synapses)}")


valid_synapse_color = "yellow"
error_color = "red"
scatters_list = []
scatters_colors = []

if len(err_synapses) > 0:
    scatters_list.append(err_synapses)
    scatters_colors.append(error_color)
if len(non_err_synapses) > 0:
    scatters_list.append(non_err_synapses)
    scatters_colors.append(valid_synapse_color)
    
if len(scatters_list) > 0:
    nviz.plot_objects(main_mesh=valid_mesh,
              meshes=[error_submesh],
                      mesh_alpha=1,
              meshes_colors=[error_color],
            scatters=[err_synapses,non_err_synapses],
             scatters_colors=[error_color,valid_synapse_color],
                     scatter_size=0.2)

else:
    nviz.plot_objects(main_mesh=valid_mesh,
             meshes=error_submesh,
              meshes_colors=error_color,
             mesh_alpha=1)


Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691135740225387
n_synapses = 2162, n_errored_synapse = 118


# Write the Results to a Datajoint Table

In [ ]:
@schema
class ValidationStats(dj.Manual):
    definition="""
    -> minnie.ManualProofreadIDs
    ---
    new_segment_id: bigint unsigned
    
    true_positive_presyn: int
    false_positive_presyn: int
    true_negative_presyn: int
    false_negative_presyn: int

    precision_presyn=NULL: double
    recall_presyn=NULL: double
    f1_presyn=NULL: double

    true_positive_postsyn: int
    false_positive_postsyn: int
    true_negative_postsyn: int
    false_negative_postsyn: int

    precision_postsyn=NULL: double
    recall_postsyn=NULL: double
    f1_postsyn=NULL: double

    true_positive_total : int
    false_positive_total : int
    true_negative_total : int
    false_negative_total : int

    precision_total=NULL: double
    recall_total=NULL: double
    f1_total=NULL: double

    """

In [ ]:
info_key = dict(segment_id=segment_id,
               new_segment_id=v7_id,
               
                true_positive_presyn = len(true_positive_presyn),
                false_positive_presyn = len(false_positive_presyn),
                true_negative_presyn= len(true_negative_presyn),
                false_negative_presyn = len(false_negative_presyn),

                precision_presyn = presyn_scores["precision"],
                recall_presyn = presyn_scores["recall"],
                f1_presyn = presyn_scores["f1"],

                true_positive_postsyn = len(true_positive_postsyn),
                false_positive_postsyn = len(false_positive_postsyn),
                true_negative_postsyn = len(true_negative_postsyn),
                false_negative_postsyn = len(false_negative_postsyn),

                precision_postsyn = postsyn_scores["precision"],
                recall_postsyn = postsyn_scores["recall"],
                f1_postsyn = postsyn_scores["f1"],

                true_positive_total= len(true_positive_total),
                false_positive_total = len(false_positive_total),
                true_negative_total = len(true_negative_total),
                false_negative_total = len(false_negative_total),

                precision_total = total_scores["precision"],
                recall_total=total_scores["recall"],
                f1_total = total_scores["f1"],
                
                
               )

ValidationStats.insert1(info_key,skip_duplicates=True)

In [ ]:
ValidationStats()

In [ ]:
TP,FP,FN = ValidationStats.fetch("true_positive_total","false_positive_total","false_negative_total")
total_dict = dict(type="presyn and postsyn")
total_stats = calculate_scores(TP = np.sum(TP),FP = np.sum(FP),FN = np.sum(FN))
total_dict.update(total_stats)
total_dict

In [ ]:
TP_pre,FP_pre,FN_pre = ValidationStats.fetch("true_positive_presyn","false_positive_presyn","false_negative_presyn")
presyn_dict = dict(type="presyn")
presyn_stats = calculate_scores(TP = np.sum(TP_pre),FP = np.sum(FP_pre),FN = np.sum(FN_pre))
presyn_dict.update(presyn_stats)
presyn_dict

In [ ]:
TP_post,FP_post,FN_post = ValidationStats.fetch("true_positive_postsyn","false_positive_postsyn","false_negative_postsyn")
postsyn_dict = dict(type="postsyn")
postsyn_stats = calculate_scores(TP = np.sum(TP_post),FP = np.sum(FP_post),FN = np.sum(FN_post))
postsyn_dict.update(postsyn_stats)
postsyn_dict

# Making the Figure for the Stats

In [ ]:
import pandas as pd

In [ ]:
pd = 